# Convert TIFF to OME-TIFF

## Motivation

Vitessce can load imaging data from OME-TIFF or OME-NGFF format. 

[OME-TIFF](https://docs.openmicroscopy.org/ome-model/5.6.3/ome-tiff/) is a file format that combines a TIFF image with metadata defined in XML format. The metadata must conform to the OME-XML specification. OME stands for Open Microscopy Environment, and OME-XML defines standard metadata fields for bioimaging and microscopy data. For instance, channels can be named according to the protein marker targeted.

In [9]:
from tifffile import imread
from os.path import join
from vitessce.data_utils import rgb_img_to_ome_tiff

## Load the original TIFF

The `imread` function from the `tifffile` package can be used to read the data in the original (non-OME) TIFF into a multidimensional NumPy array.

In this case, the array is 3-dimensional because the image has an X-axis, a Y-axis, and a channel (color) axis.

In [10]:
img_arr = imread(join('raw_data', 'dog.tiff'))
img_arr.shape

(500, 750, 4)

## Transpose and select channels of interest

We want to transpose the array so that the channel axis is first:

In [11]:
img_arr = img_arr.transpose((2, 0, 1))
img_arr.shape

(4, 500, 750)

This image has four color channels (R, G, B, A). We can discard the A (alpha) channel:

In [12]:
img_arr = img_arr[0:3, :, :]
img_arr.shape

(3, 500, 750)

## Save as an OME-TIFF

In this case, the image is RGB so we use `rgb_img_to_ome_tiff`. For multiplexed images, see [multiplex_img_to_ome_tiff](https://vitessce.github.io/vitessce-python/api_data.html#vitessce.data_utils.ome.multiplex_img_to_ome_tiff).

In [13]:
rgb_img_to_ome_tiff(
    img_arr,
    output_path=join('processed_data', 'dog.ome.tif'),
    img_name='Dog',
    axes="CYX",
)

## Convert to a pyramidal OME-TIFF using bfconvert

In [14]:
!~/software/bftools/bfconvert -overwrite -tilex 128 -tiley 128 -pyramid-resolutions 2 -pyramid-scale 2 -compression LZW ./processed_data/dog.ome.tif ./processed_data/dog.pyramid.ome.tif
# For larger images:
# !~/software/bftools/bfconvert -tilex 512 -tiley 512 -pyramid-resolutions 6 -pyramid-scale 2 -compression LZW ./processed_data/dog.ome.tif ./processed_data/dog.pyramid.ome.tif

./processed_data/dog.ome.tif
OMETiffReader initializing ./processed_data/dog.ome.tif
Reading IFDs
Populating metadata
[OME-TIFF] -> ./processed_data/dog.pyramid.ome.tif [OME-TIFF]
Tile size = 128 x 128
	Converted 3/3 planes (100%)
Tile size = 128 x 128
	Converted 3/3 planes (100%)
[done]
1.487s elapsed (2.3333333+122.666664ms per plane, 701ms overhead)
